In [1]:
import pandas as pd
import numpy as np
import random
from quantnote import BacktestEngine, Strategy

In [2]:
db_path = '/Users/j.nahmgoong/Documents/git/DB/MainDB.db'

engine = BacktestEngine(db_path)

Loading DB...
DB loaded in 46.31 seconds
[initialize] is executed in 55.41 seconds


In [3]:
non_cap_norm = ['tbvps',
'sps',
'ros',
'roe',
'roa',
'ps1',
'ps',
'price',
'prefdivis',
'pe1',
'pe',
'pb',
'payoutratio',
'fcfps',
'epsdil',
'eps',
'dps',
'divyield',
'de',
'currentratio',
'bvps',
'marketcap']

In [4]:
input_list = list(engine.cache['fundamentals']['AAPL'].columns[5:])
unary_list = ['zscore']
binary_list = ['B:add', 'B:sub', 'B:mul', 'B:div']

result_df = pd.DataFrame({
    'sharpe':[],
    'cagr':[],
    'vol':[],
    'tree':[],
})

len(input_list)

105

In [5]:
def create_tree(input_list, binary_list, max_depth):        
    tree = []
    score = 1
    depth = 0
    while True:
        if score == 0:
            return tree
            
        if depth < max_depth:
            action = random.sample(binary_list, 1)[0]
            depth += 1
            score += 1
        else:
            action = random.sample(input_list, 1)[0]
            score -= 1
            
        tree.append(action)
    
    
for _ in range(10):
    tree = create_tree(input_list, binary_list, 0)
    print(tree)
    tree = create_tree(input_list, binary_list, 1)
    print(tree)
    tree = create_tree(input_list, binary_list, 2)
    print(tree)
    

['ebitdausd']
['B:add', 'dps', 'ebt']
['B:mul', 'B:div', 'gp', 'liabilitiesnc', 'liabilities']
['ncf']
['B:sub', 'revenueusd', 'intexp']
['B:div', 'B:div', 'ebit', 'netmargin', 'ncfi']
['capex']
['B:mul', 'workingcapital', 'ebitusd']
['B:sub', 'B:sub', 'ncfx', 'netinccmnusd', 'roic']
['roa']
['B:div', 'sharesbas', 'eps']
['B:div', 'B:mul', 'ebitdamargin', 'liabilitiesnc', 'assetsnc']
['equityusd']
['B:sub', 'ebitdamargin', 'sps']
['B:add', 'B:sub', 'opinc', 'equityavg', 'currentratio']
['de']
['B:mul', 'cashnequsd', 'ncfdiv']
['B:add', 'B:add', 'ebitusd', 'ps1', 'de']
['sharefactor']
['B:sub', 'evebit', 'opinc']
['B:add', 'B:div', 'cashnequsd', 'revenueusd', 'sps']
['fxusd']
['B:add', 'ev', 'netinccmn']
['B:div', 'B:sub', 'intexp', 'liabilities', 'ncfdebt']
['ncff']
['B:add', 'sgna', 'payables']
['B:div', 'B:add', 'depamor', 'assetturnover', 'investmentsc']
['pe']
['B:add', 'accoci', 'investmentsnc']
['B:mul', 'B:div', 'fcfps', 'debtc', 'evebitda']


In [6]:
class myStrategy(Strategy):
    def __init__(self, tree):
        super().__init__()
        self.tree = tree
        
        print(self.tree)
        
    def compute_factor_from_tree(self, universe_list):
        tree = self.tree.copy()
        ops = []
        inputs = []

        while tree:
            x = tree.pop()

            if x[0]=='B': 
                ops.append(x)
            else:
                x = self.compute_factor_series(universe_list, x, type='rank')
                inputs.append(x)

            if len(inputs) >= 2 and len(ops) >= 1:
                x1 = inputs.pop()
                x2 = inputs.pop()
                binary = ops.pop()

                if binary == 'B:add': x = x1 + x2
                elif binary == 'B:sub': x = x1 - x2
                elif binary == 'B:mul': x = x1 * x2
                elif binary == 'B:div': x = x1 / x2
                    
                inputs.append(x)
                
        assert len(ops) == 0 and len(inputs) == 1
        x = inputs.pop()
        return x

    
    def compute_target(self, universe_list):
        target_weight = { }
        
        factor_series = self.compute_factor_from_tree(universe_list)
        
        factor_series = self.transform_series(factor_series, 'rank', universe_list)-0.5
        
        for ticker in factor_series.index:
            weight = factor_series[ticker]
            if weight > 0:
                target_weight[ticker] = weight
                
        assert len(target_weight) >= 25
        
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            if ftype in non_cap_norm:
                x = self.get_value('fundamentals',ticker,ftype)
            else:
                x = self.get_value('fundamentals',ticker,ftype)/self.get_value('fundamentals',ticker,'marketcap')
            return x
        

In [ ]:
while True:
    try:
        print('\n')
        depth = random.sample([0,1,2],1)[0]
        tree = create_tree(input_list, binary_list, depth)
        mys = myStrategy(tree)
        engine.run_backtest(mys, '2010-01-01','2021-08-31',period='Q')
        engine.stat(bench=None)

        print(engine.stat_dic)

        sharpe = engine.stat_dic['sharpe']
        cagr = engine.stat_dic['cagr']
        vol = engine.stat_dic['vol']
        
        result_df = result_df.append({
            'sharpe':sharpe,
            'cagr':cagr,
            'vol':vol,
            'tree':tree
        },ignore_index=True)
        
        result_df.to_parquet('result_tree.parquet')
    except:
        print('passed!')
        pass



['B:sub', 'B:div', 'roic', 'ncfdebt', 'sps']
Backtest period: 2010-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:3.889 / time elapsed:210.7 ===
[run_backtest] is executed in 212.35 seconds
{'cagr': 0.11628013412888313, 'vol': 0.17507298287224776, 'sharpe': 0.664180915988241}


['debtc']
Backtest period: 2010-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.807 / time elapsed:187.7 ===
[run_backtest] is executed in 189.11 seconds
{'cagr': 0.08843040579188055, 'vol': 0.1696761418366825, 'sharpe': 0.5211717147422943}


['evebit']
Backtest period: 2010-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:4.334 / time elapsed:207.9 ===
[run_backtest] is executed in 209.37 seconds
{'cagr': 0.12550048174025694, 'vol': 0.17905622121352482, 'sharpe': 0.7008998675929691}


['B:sub', 'B:div', 'ncfx', 'deposits', 'marketcap']
Backtest period: 2010-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-0